<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ
</h1>

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pprint

%matplotlib inline

np.random.seed(42)

Представим, что у нас есть датасет с признаками, которые сильно коррелируют, например, вес и рост для детей дошкольного возраста.

In [ ]:
N_SAMPLES = 50

def noise(delta, n=N_SAMPLES):
    return delta * np.random.randn(n)

age = np.random.randint(low=3, high=8, size=N_SAMPLES)
weight = np.round(3 + age * 3 + noise(2))
height = np.round(30 + 4 * weight + noise(6))

pprint.pprint(list(zip(age, weight, height)))

Таким образом, мы имеем следующую систему уравнений (без учета шума):

$$
age = \frac{1}{3} \cdot weight - 1
$$

$$
height = 4 \cdot weight + 30
$$

Откуда получаем:

$$
age = \frac{1}{3} \cdot \frac{height - 30}{4} - 1 = \frac{1}{12} \cdot height - 3.5
$$

In [ ]:
plt.scatter(weight, age)
plt.title('Зависимость возраста от веса')
plt.xlabel('Вес, кг')
plt.ylabel('Возраст, лет')
plt.show()

In [ ]:
plt.scatter(height, age)
plt.title('Зависимость возраста от роста')
plt.xlabel('Рост, см')
plt.ylabel('Возраст, лет')
plt.show()

In [ ]:
# добавим цвета в зависимости от возраста
plt.scatter(weight, height, c=age)
plt.title('Зависисмость роста от веса')
plt.xlabel('Вес, кг')
plt.ylabel('Рост, см')
plt.colorbar()
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

X = np.stack([weight, height], axis=1)
y = age
reg = LinearRegression()
reg.fit(X, y)

reg.coef_, reg.intercept_

Таким образом, мы получили уравнение:

$$
age = 0.31 \cdot weight - 0.012 \cdot height + 0.70
$$

Подставим $height = 4 \cdot weight + 30$:

$$
age = 0.26 \cdot weight + 0.34
$$

$$
age = 0.065 \cdot height - 1.62
$$

Как мы видим, тангенсы углов наклона близки к исходно заданным ($0.333$ и $0.083$ соответственно).

Добавим к нашему датасету один немного отличающийся образец `(7, 15, 100)` - либо выпускника детсада небольших размеров, либо ребенка с неправильно записанным возрастом:

In [ ]:
X_ = np.append(X, [[15, 100]], axis=0)
y_ = np.append(y, [7], axis=0)

reg2 = LinearRegression()
reg2.fit(X_, y_)

reg2.coef_, reg2.intercept_

Благодаря наличию скореллированных переменных, модель сильно поменяла коэффициенты, хотя мы добавили к выборке из $50$ образцов всего лишь один дополнительный экземпляр. При этом фактически сама модель осталась практически идентичной предыдущей:

$$
age = 0.22 \cdot weight + 0.009 \cdot height + 0.27
$$

$$
age = 0.26 \cdot weight + 0.54
$$

$$
age = 0.064 \cdot height - 1.38
$$

Посмотрим, что произойдет в аналогичной ситуации для модели с одной переменной:

In [ ]:
reg3 = LinearRegression()
reg3.fit(weight.reshape(-1, 1), age)

reg3.coef_, reg3.intercept_

In [ ]:
reg4 = LinearRegression()
reg4.fit(np.append(weight, [15]).reshape(-1, 1), np.append(age,[7]))

reg4.coef_, reg4.intercept_

Посчитаем изменение веса для веса (pun not intended) для разных моделей в процентах:

In [ ]:
def bias(x, y):
    return abs(x - y) / min(x, y) * 100

In [ ]:
bias(reg.coef_[0], reg2.coef_[0])

In [ ]:
bias(reg3.coef_[0], reg4.coef_[0])

Таким образом, мы видим, что веса для моделей, в которых признаки сильно скоррелированы, очень неустойчивы.